In [2]:
import dash
from dash import dash_table
from dash import dcc
from dash import html
from dash.dependencies import Output, Input
import plotly.express as px
import plotly.graph_objects as go
import dash_bootstrap_components as dbc

import pandas as pd

import sqlite3
import datetime as dt

from pandasql import sqldf
pysqldf=lambda q :sqldf(q,globals())

import json



In [3]:
conn=sqlite3.connect('CovidIndia.sqlite')
Data=pd.read_sql_query("SELECT *  FROM Data",conn)

State=pd.read_sql_query("SELECT *  FROM State",conn)

State_Data=pd.read_sql_query("SELECT *  FROM State_Data",conn)

#df=pd.read_sql_query("SELECT Date,Confirmed, Recovered, Deceased, Tested,Vaccinated, State_Name, State_Area,State_Population  FROM Data LEFT JOIN State_Data ON Data.state_id=State_Data.state_id",conn)
#df.to_excel("Test.xlsx")
#Date,Confirmed, Recovered, Deceased, Tested,Vaccinated ,State_Population 
#df=pd.read_sql_table(conn,conn)

State_Data.head()

,state_id,state_code,State_Name,State_Area,State_Population
0,19207,UP,Uttar Pradesh,243286.0,240250000
1,10968,MH,Maharashtra,307713.0,124184000
2,2343,BR,Bihar,94163.0,110020000
3,20410,WB,West Bengal,88752.0,96320000
4,12724,MP,Madhya Pradesh,308252.0,85344000


In [20]:
Joined_DF=pysqldf("Select state_code, Date,Tested,Vaccinated1,Vaccinated2,State_Name,State_Area,State_Population from Data left join State on Data.state_id=State.id left join State_Data on Data.state_id=State_Data.state_id")
Joined_DF["Date"]=pd.DatetimeIndex(Joined_DF['Date'])
Joined_DF.dtypes

#changing names to make it compatible for geojson
Joined_DF["State_Name"].replace({"Andaman and Nicobar Islands":"Andaman & Nicobar Island","Arunachal Pradesh":"Arunanchal Pradesh",
                               "Daman and Diu":"Daman & Diu","Delhi":"NCT of Delhi","Jammu and Kashmir":"Jammu & Kashmir",
                               "Pondicherry":"Puducherry",None:"Jammu & Kashmir"},inplace=True)

In [17]:
Joined_DF.head()

,state_code,Date,Tested,Vaccinated1,Vaccinated2,State_Name
0,AN,2020-03-26,0,0,0,Andaman & Nicobar Island
1,AN,2020-03-27,0,0,0,Andaman & Nicobar Island
2,AN,2020-03-28,0,0,0,Andaman & Nicobar Island
3,AN,2020-03-29,0,0,0,Andaman & Nicobar Island
4,AN,2020-03-30,0,0,0,Andaman & Nicobar Island


In [5]:
Summary_Table=pysqldf("Select State_Name,sum(Vaccinated1) as 'Partially_Vax', sum(Vaccinated2) as 'Fully_Vax'"
           "from Joined_DF group by State_Name")
Summary_Table

,State_Name,Partially_Vax,Fully_Vax
0,Andaman & Nicobar Island,294001,200157
1,Andhra Pradesh,32976969,20375181
2,Arunanchal Pradesh,771875,534486
3,Assam,20172463,8068795
4,Bihar,49874828,18346781
5,Chandigarh,926035,546981
6,Chhattisgarh,14851682,7343273
7,Daman & Diu,660753,370255
8,Goa,1262568,911114
9,Gujarat,44735217,25972387


In [18]:
#building geojson file
india_states = json.load(open("states_india.geojson", "r"))
state_id_map = {}
for feature in india_states["features"]:
    feature["id"] = feature["properties"]["state_code"]
    state_id_map[feature["properties"]["st_nm"]] = feature["id"]

In [32]:
list_of_states=sorted(Joined_DF["State_Name"].unique()) #input is an array
list_of_states.append("India")

list_of_states2=list_of_states.copy()
list_of_states2.remove("India")

# https://www.bootstrapcdn.com/bootswatch/
app = dash.Dash(__name__,external_stylesheets=[dbc.themes.BOOTSTRAP], 
               meta_tags=[{'name': 'viewport',
                            'content': 'width=device-width, initial-scale=1.0'}] #meta_tags is to make the app mobile compatible
                          ) #external_stylesheets is where you declare the theme(DARKLY here)

# Layout section: Bootstrap (https://hackerthemes.com/bootstrap-cheatsheet/)
app.layout=dbc.Container([
    
        dbc.Row([
            dbc.Col(html.H1("Vaccination Coverage",
                   className="text-center bg-light mb-4"), #put space to apply multiple classNames
                    width=12)
        ]),
    
        dbc.Row([
            dbc.Col([
                dcc.Dropdown(id="states_dpdwn",multi=False,value="India",
                            options=[{'label':x,'value':x}for x in list_of_states])
            ])
        ]),
    
        dbc.Row([
            dbc.Col([
                dcc.Graph(
                    id='line_graph_day',
                    figure={})
            ],width={'size':5, 'offset':1, 'order':1}),
            
            dbc.Col([
                dcc.Graph(
                    id='line_graph_cum',
                    figure={})
            ],width={'size':5, 'offset':1, 'order':1})
        ]),
    
        dbc.Row([
            dbc.Col([
                dcc.Dropdown(id="states_dpdwn2",multi=True,value=["Andaman & Nicobar Island"],
                            options=[{'label':x,'value':x}for x in list_of_states2])
            ])

        ]),
    
        dbc.Row([
            dbc.Col([
                dcc.Tabs(id="tabs",value="First_Dose",children=[
                        dcc.Tab(label="First_Dose",value="First_Dose"),
                        dcc.Tab(label="Second_Dose",value="Second_Dose")])
            ]),
                           
            dbc.Col([
                dcc.Graph(
                    id='line_graph_cum_com',
                    figure={})
            ],width={'size':10, 'offset':1, 'order':1})
        ]),
    
        dbc.Row([
            dbc.Col([
                dcc.DatePickerRange(
                id="Date_slider",
                min_date_allowed=Joined_DF["Date"].min(),
                max_date_allowed=Joined_DF["Date"].max(),
                initial_visible_month=Joined_DF["Date"].min(),
                start_date=Joined_DF["Date"].min(),
                end_date=Joined_DF["Date"].max()
        )
                
            ],width={'size':6, 'offset':3, 'order':1})
            ],justify='center'),
    
        dbc.Row([
            dbc.Col([
                dcc.Graph(
                    id='India_Map1',
                    figure={})
            ]),
            
            dbc.Col([
                dcc.Graph(
                    id='India_Map2',
                    figure={})
            ])
        ])
    
])

# Callback section: connecting the components
#callback for first two graphs
@app.callback(
    [Output('line_graph_day','figure'),
    Output('line_graph_cum','figure')],
    Input('states_dpdwn','value')
)

def update_graph1(state):
    if state=="India":
        dff=Joined_DF.groupby("Date").sum().reset_index()
        dff.insert(3,"Cum_Vax1",dff["Vaccinated1"].cumsum(),"True")
        dff.insert(5,"Cum_Vax2",dff["Vaccinated2"].cumsum(),"True")
        
        fig1=go.Figure()
        fig1.add_trace(go.Scatter(
                x=dff["Date"],
                y=dff["Vaccinated1"],
                mode='lines',
                name="First Dose"
            ))

        fig1.add_trace(go.Scatter(
                x=dff["Date"],
                y=dff["Vaccinated2"],
                mode='lines',
                name="Second Dose"                    
            ))
        

        fig2=go.Figure()
        fig2.add_trace(go.Scatter(
                x=dff["Date"],
                y=dff["Cum_Vax1"],
                mode='lines',
                name="First Dose"
            ))

        fig2.add_trace(go.Scatter(
                x=dff["Date"],
                y=dff["Cum_Vax2"],
                mode='lines',
                name="Second Dose"                    
            ))
        
        
        
    else:
        dff=Joined_DF[Joined_DF["State_Name"]==state]
        dff.insert(3,"Cum_Vax1",dff["Vaccinated1"].cumsum(),"True")
        dff.insert(5,"Cum_Vax2",dff["Vaccinated2"].cumsum(),"True")    
        
        fig1=go.Figure()
        fig1.add_trace(go.Scatter(
                x=dff["Date"],
                y=dff["Vaccinated1"],
                mode='lines',
                name="First Dose"
            ))

        fig1.add_trace(go.Scatter(
                x=dff["Date"],
                y=dff["Vaccinated2"],
                mode='lines',
                name="Second Dose"                    
            ))
        

        fig2=go.Figure()
        fig2.add_trace(go.Scatter(
                x=dff["Date"],
                y=dff["Cum_Vax1"],
                mode='lines',
                name="First Dose"
            ))

        fig2.add_trace(go.Scatter(
                x=dff["Date"],
                y=dff["Cum_Vax2"],
                mode='lines',
                name="Second Dose"                    
            ))
        

    return fig1,fig2

#callback for 3rd graph(comparing percentage coverage by day)
@app.callback(
    Output('line_graph_cum_com','figure'),
    Input('states_dpdwn2','value'),
    Input('tabs','value')
)

def update_graph2(state,tab):
    
    dff=Joined_DF[Joined_DF["State_Name"].isin(state)]
    dff.insert(3,"Cum_Vax1",dff.groupby(["State_Name"])["Vaccinated1"].cumsum(),True)
    dff.insert(5,"Cum_Vax2",dff.groupby(["State_Name"])["Vaccinated2"].cumsum(),True)
    dff.insert(4,"Vax1_Coverage",dff["Cum_Vax1"]/dff["State_Population"],True)
    dff.insert(6,"Vax2_Coverage",dff["Cum_Vax2"]/dff["State_Population"],True)

    if tab=="First_Dose":
        fig=px.line(
                dff,
                x="Date",
                y="Vax1_Coverage",
                color="State_Name")
        fig.show()
        
    elif tab=="Second_Dose":
        
        fig=px.line(
                dff,
                x="Date",
                y="Vax2_Coverage",
                color="State_Name")

    return fig

#callback for last 2 maps
@app.callback(
    Output("India_Map1",'figure'),
    Output("India_Map2","figure"),
    Input('Date_slider','start_date'),
    Input('Date_slider','end_date') 
)

def update_graph3(start_date,end_date):
        
    dff=Joined_DF.drop(columns=["State_Area","State_Population"])

    mask=(dff["Date"]>start_date) & (dff["Date"]<end_date)
    dff=dff.loc[mask]
    
    dff2=dff.groupby("State_Name").sum().reset_index()
    dff3=pysqldf("Select dff2.State_Name,Tested,Vaccinated1,Vaccinated2, State_Population from dff2 left join State_Data on dff2.State_Name=State_Data.State_Name")
    
    dff3["id"]=dff2["State_Name"].apply(lambda x: state_id_map[x])
    dff3.insert(3,"Vax1_Coverage",dff3["Vaccinated1"]/dff3["State_Population"],True)
    dff3.insert(5,"Vax2_Coverage",dff3["Vaccinated2"]/dff3["State_Population"],True)
    
    map1=px.choropleth(
            dff3,
            locations="id",
            geojson=india_states,
            color="Vax1_Coverage",
            hover_name="State_Name",
            hover_data=["State_Name"],
            color_continuous_scale=px.colors.diverging.RdYlGn, #https://plotly.com/python/builtin-colorscales/
            range_color=[0, 1]
    )

    map1.update_geos(fitbounds="locations", visible=False)
    
    map2=px.choropleth(
            dff3,
            locations="id",
            geojson=india_states,
            color="Vax2_Coverage",
            hover_name="State_Name",
            hover_data=["State_Name"],
            color_continuous_scale=px.colors.diverging.RdYlGn,  #https://plotly.com/python/builtin-colorscales/
            range_color=[0, 1]
        )

    map2.update_geos(fitbounds="locations", visible=False)
    
    
    return map1,map2

if __name__ == '__main__':
    app.run_server(debug=True,use_reloader=False)


Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


In [21]:
# pip install plotly 5.0.0

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement 5.0.0
ERROR: No matching distribution found for 5.0.0


In [ ]:
# k=Joined_DF.groupby("Date").sum().reset_index()
# k.insert(3,"Cum_Vax1",k["Vaccinated1"].cumsum(),"True")
# k.insert(5,"Cum_Vax2",k["Vaccinated2"].cumsum(),"True")
# k.head()
# fig1=go.Figure()
# fig1.add_trace(go.Scatter(
#         x=k["Date"],
#         y=k["Vaccinated1"],
#         mode='lines',
#         name="First Dose"
#     ))

# fig1.add_trace(go.Scatter(
#         x=k["Date"],
#         y=k["Vaccinated2"],
#         mode='lines',
#         name="Second Dose"                    
#     ))

# fig1

In [ ]:
x=["West Bengal","Kerala"]
b=Joined_DF[Joined_DF["State_Name"].isin(x)]
b.insert(3,"Cum_Vax1",b.groupby(["State_Name"])["Vaccinated1"].cumsum(),True)
b.insert(5,"Cum_Vax2",b.groupby(["State_Name"])["Vaccinated2"].cumsum(),True)
# b["Cum_Vax1"]=b.groupby(["State_Name"])["Vaccinated1"].cumsum()
# b["Cum_Vax2"]=b.groupby(["State_Name"])["Vaccinated2"].cumsum()

In [ ]:
fig=px.line(
        b,
        x="Date",
        y="Cum_Vax2",
        color="State_Name")

fig




In [ ]:
Joined_DF

In [ ]:
state=["Andaman & Nicobar Island"]
dff=Joined_DF[Joined_DF["State_Name"].isin(state)]
dff.insert(3,"Cum_Vax1",dff.groupby(["State_Name"])["Vaccinated1"].cumsum(),True)
dff.insert(5,"Cum_Vax2",dff.groupby(["State_Name"])["Vaccinated2"].cumsum(),True)
dff.insert(4,"Vax1_Coverage",dff["Cum_Vax1"]/dff["State_Population"],True)
dff.insert(6,"Vax2_Coverage",dff["Cum_Vax2"]/dff["State_Population"],True)
# dff=dff.style.format({"Vax1_Coverage":"{:.2%}","Vax2_Coverage":"{:.2%}"})
dff

In [ ]:
a=Joined_DF
# mask=(dff["Date"]>start_date) & (dff["Date"]<end_date)
# dff=dff.loc[mask]

# a.drop(columns=["State_Area","State_Population"],inplace=True)
a=a.groupby("State_Name").sum().reset_index()
b=pysqldf("Select a.State_Name,Tested,Vaccinated1,Vaccinated2, State_Population from a left join State_Data on a.State_Name=State_Data.State_Name")

b.insert(3,"Vax1_Coverage",b["Vaccinated1"]/b["State_Population"])
b.insert(5,"Vax2_Coverage",b["Vaccinated2"]/b["State_Population"])
b



In [8]:
# a=Joined_DF.groupby("State_Name").sum().reset_index()["State_Name"].to_list()
# b=State_Data["State_Name"].to_list()

# not_in_a=[]
# not_in_b=[]


# for i in b:
#     if i not in a:
#         not_in_a.append(i)

        
# for i in a:
#     if i not in b:
#         not_in_b.append(i)
        
# not_in_a, not_in_b


In [19]:
sorted(Joined_DF["State_Name"].unique())

TypeError: '<' not supported between instances of 'NoneType' and 'str'

In [38]:
list_of_states=Joined_DF["State_Name"].unique()#input is an array
a=list_of_states.tolist()
a.append("India")

b=a.copy()
b.remove("India")
a

['Andaman & Nicobar Island',
 'Andhra Pradesh',
 'Arunanchal Pradesh',
 'Assam',
 'Bihar',
 'Chandigarh',
 'Chhattisgarh',
 'NCT of Delhi',
 'Daman & Diu',
 'Goa',
 'Gujarat',
 'Himachal Pradesh',
 'Haryana',
 'Jharkhand',
 'Jammu & Kashmir',
 'Karnataka',
 'Kerala',
 'Lakshadweep',
 'Maharashtra',
 'Meghalaya',
 'Manipur',
 'Madhya Pradesh',
 'Mizoram',
 'Nagaland',
 'Odisha',
 'Punjab',
 'Puducherry',
 'Rajasthan',
 'Sikkim',
 'Telangana',
 'Tamil Nadu',
 'Tripura',
 'Uttar Pradesh',
 'Uttarakhand',
 'West Bengal',
 'India']

In [26]:
dff=Joined_DF.drop(columns=["State_Area","State_Population"])

# mask=(dff["Date"]>start_date) & (dff["Date"]<end_date)
# dff=dff.loc[mask]

dff2=dff.groupby("State_Name").sum().reset_index()
dff3=pysqldf("Select dff2.State_Name,Tested,Vaccinated1,Vaccinated2, State_Population from dff2 left join State_Data on dff2.State_Name=State_Data.State_Name")

dff3["id"]=dff2["State_Name"].apply(lambda x: state_id_map[x])
dff3.insert(3,"Vax1_Coverage",dff3["Vaccinated1"]/dff3["State_Population"],True)
dff3.insert(5,"Vax2_Coverage",dff3["Vaccinated2"]/dff3["State_Population"],True)
dff3

TypeError: insert() got an unexpected keyword argument 'inplace'